In [1]:
import numpy as np
import cupy as cp

from tqdm.notebook import tqdm
from time import time

from toric import *
from pymatching import Matching

In [2]:
# Simulation parameters

N = 100 # Shots
L = 100 # Lattice size
p_error = 0.004 # Error probability per spin
η = 0.1 # Smoothing paramter for Jacobi method
c = 16 # "Field velocity" - number of field updates per cycle
T = L # Epochs

matching = Matching(pcm(L))

In [3]:
state = State(N, L)
for _ in range(T):
    if p_error > 0:
        state.add_errors(p_error)
    for _ in range(c):
        state.update_field(η)
    state.update_anyon()
x_correction, y_correction = mwpm(matching, state.q)
cp.mean(logical_error(state.x_error ^ x_correction, state.y_error ^ y_correction))

array(0.)

In [4]:
timings = np.zeros((100, 6))
for i in tqdm(range(100)):
    t0 = time()
    state = State(N, L)
    timings[i, 0] = time() - t0
    for _ in range(T):
        t0 = time()
        state.add_errors(p_error)
        timings[i, 1] += (time() - t0) / T
        for _ in range(c):
            t0 = time()
            state.update_field(η)
            timings[i, 2] += (time() - t0) / T / c
        t0 = time()
        state.update_anyon()
        timings[i, 3] += (time() - t0) / T
    t0 = time()
    x_correction, y_correction = mwpm(matching, state.q)
    timings[i, 4] = time() - t0
    t0 = time()
    cp.mean(logical_error(state.x_error ^ x_correction, state.y_error ^ y_correction))
    timings[i, 5] = time() - t0
timings = timings.mean(axis = 0)
print(timings)

  0%|          | 0/100 [00:00<?, ?it/s]

[0.0004322  0.00106506 0.00019237 0.00603618 0.05899534 0.00115001]


In [4]:
%timeit state = State(N, L)
%timeit state.add_errors(p_error)
%timeit state.update_field(η)
%timeit state.update_anyon()
%timeit x_correction, y_correction = mwpm(matching, state.q)
%timeit np.mean(logical_error(state.x_error ^ x_correction, state.y_error ^ y_correction))

268 μs ± 26.3 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
6.47 ms ± 68.7 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.72 ms ± 40.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
16.6 ms ± 659 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
6.89 s ± 64.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.21 ms ± 98.4 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
